In [5]:
#LLM和TR都因為和Tensorflow 有一些問題啊啊啊
#use finetune
#import unsloth#no.1
import requests
import zipfile
import os

# 数据集下载 URL（从官方网站获取）
url = "https://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip"
output_dir = "cornell_movie_dialogs_corpus"

# 下载并保存 zip 文件
response = requests.get(url)
zip_file_path = "cornell_movie_dialogs_corpus.zip"
with open(zip_file_path, "wb") as f:
    f.write(response.content)

# 解压 zip 文件
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_dir)

# 删除 zip 文件
os.remove(zip_file_path)

print(f"Dataset downloaded and extracted to '{output_dir}'")


Dataset downloaded and extracted to 'cornell_movie_dialogs_corpus'


In [6]:
import json

def load_lines(file_path):
    # 加载并解析 movie_lines.txt 文件
    lines = {}
    with open(file_path, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                lines[parts[0]] = parts[4]  # 保存 lineID 和台词
    return lines

def load_conversations(file_path, lines):
    # 加载并解析 movie_conversations.txt 文件，并将对话拼接起来
    conversations = []
    with open(file_path, 'r', encoding='iso-8859-1') as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            line_ids = eval(parts[-1])
            conversation = [lines[line_id] for line_id in line_ids if line_id in lines]
            conversations.append(conversation)
    return conversations

def prepare_dataset(conversations, output_path):
    # 将每段对话拼接为一个连续的文本片段
    with open(output_path, 'w', encoding='utf-8') as f:
        for conversation in conversations:
            for line in conversation:
                f.write(line + "\n")
            f.write("\n")  # 每段对话之间留一个空行

# 处理数据
lines = load_lines("/home/kaia/BasicTR/cornell movie-dialogs corpus/movie_lines.txt")
conversations = load_conversations("/home/kaia/BasicTR/cornell movie-dialogs corpus/movie_conversations.txt", lines)
prepare_dataset(conversations, "prepared_conversations.txt")


In [7]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq, GPT2Tokenizer, GPT2LMHeadModel
#from unsloth import is_bfloat16_supported


def load_and_prepare_data():
    dataset = load_dataset("text", data_files={"train": "prepared_conversations.txt"})
    return dataset["train"]

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [11]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import DataLoader

# 定义 GPT-2 微调器类
class GPT2FineTuner:
    def __init__(self, model_name):
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        # 设置 pad_token 为 eos_token
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def forward(self, input_ids, attention_mask=None, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def tokenize_function(self, examples):
        # 对文本进行编码，并设置 labels 与 input_ids 相同
        encoding = self.tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
        encoding["labels"] = encoding["input_ids"].copy()  # 设置 labels
        return encoding
'''
# 加载和处理数据集
def load_and_prepare_data():
    dataset = load_dataset("text", data_files={"train": "prepared_conversations.txt"})
    return dataset["train"]
'''
def load_and_prepare_data(finetuner, batch_size=16, num_workers=4):
    # 加载数据集
    dataset = load_dataset("text", data_files={"train": "prepared_conversations.txt"})["train"]
    # 处理数据
    tokenized_dataset = dataset.map(finetuner.tokenize_function, batched=True)

    # 使用 DataLoader 来加载数据集
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=lambda x: {
            'input_ids': torch.tensor([f['input_ids'] for f in x], dtype=torch.long),
            'attention_mask': torch.tensor([f['attention_mask'] for f in x], dtype=torch.long),
            'labels': torch.tensor([f['input_ids'] for f in x], dtype=torch.long)  # 设置 labels
        }
    )
    return dataloader

model_name = "gpt2"
finetuner = GPT2FineTuner(model_name)
# 调用函数加载数据
dataloader = load_and_prepare_data(finetuner, batch_size=16, num_workers=4)
#tokenized_dataset = dataset.map(finetuner.tokenize_function, batched=True)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./finetune_gpt2",
    per_device_train_batch_size=16,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",  # 日志保存目录
    save_steps=500,  # 每 500 步保存一次模型
    logging_steps=100,  # 每 100 步打印日志
)
# 自定义 Trainer 类来支持自定义 DataLoader
class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        return dataloader

# 初始化 Trainer
trainer = CustomTrainer(
    model=finetuner.model,
    args=training_args,
    train_dataset=None,  # 因为我们重写了 get_train_dataloader，不需要传递 train_dataset
)

# 开始训练
trainer.train()
'''
# 初始化 Trainer
trainer = Trainer(
    model=finetuner.model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# 开始训练
trainer.train()
'''

  0%|          | 0/24222 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
'''
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader
import torch
from datasets import load_dataset
from torch.optim import AdamW

# 定义 GPT-2 微调器类
class GPT2FineTuner:
    def __init__(self, model_name):
        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def tokenize_function(self, examples):
        encoding = self.tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
        encoding["labels"] = encoding["input_ids"].copy()
        return encoding

# 加载和处理数据集，并使用 DataLoader
def load_and_prepare_data(batch_size=4, num_workers=0):
    # 加载数据集
    dataset = load_dataset("text", data_files={"train": "prepared_conversations.txt"})["train"]
    # 处理数据
    tokenized_dataset = dataset.map(finetuner.tokenize_function, batched=True)
    
    # 使用 DataLoader 来加载数据集
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=lambda x: {
            'input_ids': torch.tensor([f['input_ids'] for f in x], dtype=torch.long),
            'attention_mask': torch.tensor([f['attention_mask'] for f in x], dtype=torch.long),
            'labels': torch.tensor([f['labels'] for f in x], dtype=torch.long)
        }
    )
    return dataloader

# 加载模型
model_name = "gpt2"
finetuner = GPT2FineTuner(model_name)
dataloader = load_and_prepare_data(batch_size=16, num_workers=2)

# 训练设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
finetuner.model.to(device)
optimizer = AdamW(finetuner.model.parameters(), lr=5e-5)

# 自定义训练循环
for epoch in range(1):  # 设置 epoch 数量
    finetuner.model.train()
    for batch in dataloader:
        # 将 batch 数据传到 GPU（如果可用）
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 前向传播
        outputs = finetuner.model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")
'''


'\nfrom transformers import GPT2LMHeadModel, GPT2Tokenizer\nfrom torch.utils.data import DataLoader\nimport torch\nfrom datasets import load_dataset\nfrom torch.optim import AdamW\n\n# 定义 GPT-2 微调器类\nclass GPT2FineTuner:\n    def __init__(self, model_name):\n        self.model = GPT2LMHeadModel.from_pretrained(model_name)\n        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)\n        self.tokenizer.pad_token = self.tokenizer.eos_token\n\n    def tokenize_function(self, examples):\n        encoding = self.tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)\n        encoding["labels"] = encoding["input_ids"].copy()\n        return encoding\n\n# 加载和处理数据集，并使用 DataLoader\ndef load_and_prepare_data(batch_size=4, num_workers=0):\n    # 加载数据集\n    dataset = load_dataset("text", data_files={"train": "prepared_conversations.txt"})["train"]\n    # 处理数据\n    tokenized_dataset = dataset.map(finetuner.tokenize_function, batched=True)\n    \n    # 使用 DataL

In [13]:
import gc
torch.cuda.empty_cache()
gc.collect()

1540

In [4]:
def generate_text(model, tokenizer, prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


# 1. 加载预训练的 GPT-2 模型
pretrained_model_name = "gpt2"
pretrained_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name)
pretrained_model = GPT2LMHeadModel.from_pretrained(pretrained_model_name)

# 生成预训练模型的输出
prompt = "How are you doing today?"
pretrained_output = generate_text(pretrained_model, pretrained_tokenizer, prompt)
print("Output from Pre-trained Model:")
print(pretrained_output)

# 加载微调后的模型
finetuned_model_path = "./unsloth_gpt2"
finetuned_model = GPT2LMHeadModel.from_pretrained(finetuned_model_path)
finetuned_tokenizer = GPT2Tokenizer.from_pretrained(finetuned_model_path)

# 生成微调后模型的输出
finetuned_output = generate_text(finetuned_model, finetuned_tokenizer, prompt)
print("\nOutput from Fine-tuned Model:")
print(finetuned_output)

# 3. 对比两个模型的输出
print("\nComparison:")
print("Pre-trained Output:", pretrained_output)
print("Fine-tuned Output:", finetuned_output)

'''
# 输入初始对话文本
input_text = "How are you doing today?"

# 对输入进行编码
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 生成对话
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
)

# 解码生成的文本
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)
'''


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output from Pre-trained Model:
How are you doing today?

I'm doing a lot of work. I'm working on a new book. And I've got a couple of new projects coming up.
...
 (Laughs.)



What's your


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./unsloth_gpt2.

In [14]:
'''
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, TFGPT2LMHeadModel
import torch
from unsloth import FastLanguageModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载预训练模型和分词器
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
#model = TFGPT2LMHeadModel.from_pretrained(model_name)
model.to(device)

# 加载和准备数据集
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

def data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

dataset = load_dataset("prepared_conversations.txt", tokenizer)
collator = data_collator(tokenizer)

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# 进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=dataset,
)

trainer.train()
'''


'\nfrom transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, TFGPT2LMHeadModel\nimport torch\nfrom unsloth import FastLanguageModel\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n# 加载预训练模型和分词器\nmodel_name = "gpt2"\ntokenizer = GPT2Tokenizer.from_pretrained(model_name)\nmodel = GPT2LMHeadModel.from_pretrained(model_name)\n#model = TFGPT2LMHeadModel.from_pretrained(model_name)\nmodel.to(device)\n\n# 加载和准备数据集\ndef load_dataset(file_path, tokenizer, block_size=128):\n    return TextDataset(\n        tokenizer=tokenizer,\n        file_path=file_path,\n        block_size=block_size,\n    )\n\ndef data_collator(tokenizer):\n    return DataCollatorForLanguageModeling(\n        tokenizer=tokenizer,\n        mlm=False,\n    )\n\ndataset = load_dataset("prepared_conversations.txt", tokenizer)\ncollator = data_collator(tokenizer)\n\n# 设置训练参数\ntraining_args = TrainingArguments(\n    output_dir="./gpt